In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
%cd /content/gdrive/MyDrive/collab/tumai/testmodel

In [ ]:
!ls

In [ ]:
dataframe = pd.read_csv("dataset_1_balanced-2.csv")
col_name_drop = [f"steps_{i}" for i in range(10)]
dataframe = dataframe.drop(columns = col_name_drop)
dataframe.pop("Unnamed: 0")
train, val, test = np.split(dataframe.sample(frac=1), [int(0.8*len(dataframe)), int(0.9*len(dataframe))])
print(len(train), 'training examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')
print(train.label.value_counts())
print(val.label.value_counts())
y_val= val.pop('label')
y_train = train.pop('label')
y_test = test.pop('label')
print(val.head())

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# val as well would be transformed with the same this is for testing
train = scaler.fit_transform(train)

In [ ]:
y_train.value_counts()

In [ ]:
print(y_train.values)

In [ ]:
from keras import Sequential
from keras.layers import Dense
import tensorflow as tf


In [ ]:
classifier = Sequential()
#First Hidden Layer
classifier.add(Dense(3, activation='relu', kernel_initializer='random_normal', input_dim=10))
#Second  Hidden Layer
classifier.add(Dense(2, activation='relu'))
#Output Layer
classifier.add(Dense(1, activation='sigmoid'))

In [ ]:
optimizer = tf.keras.optimizers.Adam()


In [ ]:
classifier.compile(optimizer =optimizer,loss='binary_crossentropy', metrics =['accuracy'])

In [ ]:
classifier.fit(train,y_train, batch_size=10, epochs=2)

In [ ]:
from torch.utils.data import DataLoader,Dataset
import torch
import torch.nn as nn

In [ ]:
class Data(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X.astype(np.float32))
        self.y = torch.from_numpy(y.astype(np.float32))
        self.len = self.X.shape[0]
       
    def __getitem__(self, index):
        return self.X[index], self.y[index]
   
    def __len__(self):
        return self.len

In [ ]:
train_data = Data(train, y_train.values)

In [ ]:
train_dataloader = DataLoader(dataset=train_data, batch_size=1, shuffle=True)

In [ ]:
import torch.nn as nn

In [ ]:
import torch

In [ ]:
class SimpleNetwork(nn.Module):
    def __init__(self):
        super().__init__()

        # set hyperparams
        self.model = nn.Sequential(
            nn.Linear(10, 4),
            nn.ReLU(),
            nn.Linear(4, 3),
            nn.ReLU(),
            nn.Linear(3, 1),
            nn.Sigmoid()

        )

    def forward(self, x):
        x = self.model(x)

        return x

In [ ]:
Net=SimpleNetwork()
learning_rate = 1e-4
criterion = nn.BCELoss()

optimizer = torch.optim.Adam(Net.parameters(), lr=learning_rate,weight_decay=0.8)

In [ ]:


l_list=[]
for epoch in range(2):
    for X, y in train_dataloader:
        # zero the parameter gradients
        optimizer.zero_grad()
        out = Net(X)
        print(out)
        loss = criterion(out, y.unsqueeze(-1))
        l_list.append(loss.item())
        loss.backward()
        optimizer.step()

In [ ]:
print(l_list)